In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import pickle
import matplotlib.dates as mdates
from datetime import datetime

ModuleNotFoundError: No module named 'pymysql'

In [ ]:
from sqlalchemy import create_engine
USER = "ciara"
PASSWORD = "Ciara123"
PORT = "3306"
DB = "dbbikes"
URI = "dbbikes.cvr6gofoxmkp.us-east-2.rds.amazonaws.com"
engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [ ]:
sql = f"""
SELECT id, number, updated_on, available_bikes, available_bike_stands
FROM dbbikes_info
WHERE updated_on is not null and updated_on >= '2021-03-24 19:25:16' and number != 507
GROUP BY id, day(updated_on)
ORDER by id desc, updated_on desc;
"""

In [ ]:
bike_df = pd.read_sql_query(sql, engine)

In [ ]:
bike_df['updated_on'] = bike_df['updated_on'].astype('datetime64[ns]')
bike_df['updated_on'] = pd.to_datetime(bike_df['updated_on'])
bike_df

In [ ]:
bike_df.dtypes

In [ ]:
bike_df.shape

In [ ]:
bike_df.tail()

In [ ]:
sql = f"""
SELECT * FROM weather;
"""

In [ ]:
weather_df = pd.read_sql_query(sql, engine)

In [ ]:
weather_df.dtypes

In [ ]:
# category    
weather_df['description'] = weather_df['description'].astype('category')    

# datetime   
weather_df['last_update'] = weather_df['last_update'].astype('datetime64[ns]')

In [ ]:
weather_df.dtypes

In [ ]:
weather_df.shape

In [ ]:
weather_df.tail()

In [ ]:
bike_df.sort_values('updated_on', inplace=True)
weather_df.sort_values('last_update', inplace=True) 

In [ ]:
df = pd.merge_asof(bike_df, weather_df, left_on='updated_on', right_on='last_update')

In [ ]:
df.shape

In [ ]:
df = df.drop(['last_update'], axis=1)

In [ ]:
df.tail(5)

In [ ]:
df = df.drop(['id_y'], axis=1)

In [ ]:
df = df.drop(['id_x'], axis=1)

In [ ]:
df.tail(5)

In [ ]:
df.shape

In [ ]:
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

In [ ]:
df = df.drop_duplicates(keep='last')
print('Duplicate remaining rows:', df[df.duplicated(keep='last')].shape[0])

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df['hourly'] = df['updated_on'].dt.hour
hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')
df = pd.concat([df, hourly_dummies], axis=1)
hour_dummies = list(hourly_dummies)
df = df.drop(['hourly'], axis=1)

df['weekday'] = df['updated_on'].dt.dayofweek

In [ ]:
print(df.head(10))
print(df.keys())

In [ ]:
stations = list(sorted(df.number.unique()))
weekdays = list(sorted(df.weekday.unique()))
models = {}

i = 0

for station in stations:
    models[station] = {}
    for weekday in weekdays:
        df1 = df[(df['number'] == station) & (df['weekday'] == weekday)]
        y = pd.DataFrame(df1, columns=['available_bikes', 'temp'])
        X = pd.DataFrame(df1, columns=['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        models[station][weekday] = LinearRegression().fit(X_train, y_train)
        model = LinearRegression().fit(X_train, y_train)
        
        train_predictions = model.predict(X_train)
        # print('Train predictions for station ', station, ": ", train_predictions)

        test_predictions = model.predict(X_test)
        # print('Test predictions for station ', station, ": ", test_predictions)

        
        r2 = metrics.r2_score(y_test, test_predictions)

        # how well does the model behave compared to a chosen r2 ? 
        if r2 > 0.5:             
            i +=1

print('Percentage of models that have outperformed r2 = 0.5:', (i /109) * 100)  # there are 109 stations
# print(i)
# print(len(stations))

# with open('model.pkl', 'wb') as handle:
#     pickle.dump(models, handle, protocol = pickle.HIGHEST_PROTOCOL)